In [1]:
import pyspark.sql.functions as f

In [2]:
df = spark.sql("select * from clickstream_de_csv")
df.show(5)
df.printSchema()

+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
 userid| offerid|countrycode| category| merchant| utcdate|rating|
+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
d581f697c19367969...|be83df9772ec47fd2...| de| 125801|4740b6c83b6e12e42...|2016-06-15 09:43:32| 0|
b6208e4f5c0ea1a5d...|b75d6ee742849a622...| de|100020213|26467f203f42913b6...|2016-06-15 09:43:36| 0|
b4dfe41dcb584aee3...|3ba463bbdb2e24eac...| de|100020213|10698b6475abd54c5...|2016-06-15 09:44:02| 0|
d581f697c19367969...|be83df9772ec47fd2...| de| 125801|4740b6c83b6e12e42...|2016-06-15 09:44:04| 0|
351163eb97a830c6c...|b75d6ee742849a622...| de|100020213|26467f203f42913b6...|2016-06-15 09:44:12| 0|
+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
only showing top 5 rows

root
-- userid: string (nullable = true)
-- offerid: string (nullable = true)
-- countrycode: string (nullable = true)
-- category: integer (nullable = true)
-- merchant: string (nullable = true)
-- utcdate: timestamp (nullable = true)
-- rating: integer (nullable = true)

In [3]:
# extract out important features and removing unneeded columns
## 'hour' is expected to be an important feature
## 'day' is expected to be an important feature
## 'week' number may not be significant (in such a short time span) but captured anyways
## 'month' month my not be significant (in such a short time span) but captured anyways
# didn't observe any German holidays during or shortly after that can have an impact on click

df = df \
  .withColumn('day', f.dayofweek('utcdate')) \
  .withColumn('week', f.weekofyear('utcdate')) \
  .withColumn('month', f.month('utcdate')) \
  .withColumn('hour', f.hour('utcdate'))
df = df.drop(*['countrycode', 'utcdate'])

df.show(10)

df.createOrReplaceTempView('impressions')

# verifying our schema

df.printSchema()

# number of partitions

print("Number of partitions for 'df': ", df.rdd.getNumPartitions())


+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
 userid| offerid| category| merchant|rating|day|week|month|hour|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
b6208e4f5c0ea1a5d...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
b4dfe41dcb584aee3...|3ba463bbdb2e24eac...|100020213|10698b6475abd54c5...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
351163eb97a830c6c...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
c1054eb52d2a47aec...|3735290a415dc236b...| 125801|8bf8f87492a799528...| 0| 4| 24| 6| 9|
8163ddf01acd57116...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
b4380e45e42d8a18d...|146d635197d4d9092...|100010713|66863da8db7e6c51b...| 0| 4| 24| 6| 9|
badefb50c4de15ff6...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
only showing top 10 rows

root
-- userid: string (nullable = true)
-- offerid: string (nullable = true)
-- category: integer (nullable = true)
-- merchant: string (nullable = true)
-- rating: integer (nullable = true)
-- day: integer (nullable = true)
-- week: integer (nullable = true)
-- month: integer (nullable = true)
-- hour: integer (nullable = true)

Number of partitions for 'df': 8

In [4]:
# looking at time frame discriptive stats

df.select(['hour', 'day', 'week', 'month', 'rating']).describe().show()

print("Number of distinct 'category': ", df.select('category').distinct().count())
print("Number of distinct 'merchant': ", df.select('merchant').distinct().count())
print("Number of distinct 'userid': ", df.select('userid').distinct().count())
print("Number of distinct 'offerid': ", df.select('offerid').distinct().count())

+-------+-----------------+------------------+------------------+-------------------+-------------------+
summary| hour| day| week| month| rating|
+-------+-----------------+------------------+------------------+-------------------+-------------------+
 count| 1919561| 1919561| 1919561| 1919561| 1919561|
 mean| 9.93217407521824|3.9443570691423715|24.504959206818643| 6.0025089069844615| 0.319952843384503|
 stddev|6.951414914815199| 1.548769034081369|0.7113425117138505|0.05002612991175306|0.46645807394142264|
 min| 0| 1| 24| 6| 0|
 max| 23| 7| 26| 7| 1|
+-------+-----------------+------------------+------------------+-------------------+-------------------+

Number of distinct 'category': 267
Number of distinct 'merchant': 738
Number of distinct 'userid': 278293
Number of distinct 'offerid': 380803

In [5]:
# Total number of clicks
df1 = df.groupby().sum('rating')
df1.show()

+-----------+
sum(rating)|
+-----------+
 614169|
+-----------+

In [6]:
# offer id that has received the max percentage of clicks
# we are assuming that the event with the most clickthrough rate and highest ad impression is most important

spark.sql('''
select offerid, sum(rating) as clicks, count(offerid) as impressions,
    sum(rating) * 100 / count(offerid) as pct
from test_de_csv
group by offerid
order by clicks desc, impressions desc, pct desc
limit 1
''' ).show(truncate=False)


+--------------------------------+------+-----------+-----------------+
offerid |clicks|impressions|pct |
+--------------------------------+------+-----------+-----------------+
0f2fcf95319f5c1e5745371351f521e5|41940 |49598 |84.55986128472922|
+--------------------------------+------+-----------+-----------------+

In [7]:
# user that has the max percentage of click
# we are assuming that the event with the most clickthrough rate and highest ad impression is most important

sample_df = df.sample(False, .5, )

spark.sql('''
select userid, sum(rating) as clicks, count(offerid) as impressions,
    sum(rating) * 100 / count(offerid) as pct
from test_de_csv
group by userid
order by clicks desc, impressions desc, pct desc
limit 1
''' ).show(truncate=False)

+----------------------------------------------------------------+------+-----------+------------------+
userid |clicks|impressions|pct |
+----------------------------------------------------------------+------+-----------+------------------+
241565de8ae842718ea3359c64246c38dc5b5335841d404d2b044a44a7cd7d23|4049 |15850 |25.545741324921135|
+----------------------------------------------------------------+------+-----------+------------------+

In [8]:
# Predict if the offer will be clicked or not

# 'userid', 'offerid' has too many categories
# 'month', 'week' has no predictive power since single or few categories
# unless we are able to group it make it smaller, we will drop it

df = df.drop(*['userid', 'offerid', 'month', 'week'])

# imports

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

# handling categorical columns

categorical_cols = ['category', 'merchant']

indexers = [StringIndexer(inputCol=col, outputCol=col+'idx') \
            .setHandleInvalid('keep') \
            for col in categorical_cols]

indexers_pipeline = Pipeline(stages=indexers)

df_indexed = indexers_pipeline.fit(df).transform(df).drop(*categorical_cols)

# creating assembler

feature_cols = df_indexed.columns[1:]

assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

df_assembled = assembler.transform(df_indexed)

df_ml = df_assembled.select(['rating', 'features'])

# Train-Test-Split

(df_train, df_test) = df_ml.randomSplit([0.6, 0.4], seed=42)

In [9]:
# random forest classifier (training set)

from pyspark.ml.classification import RandomForestClassifier as RFC

rfc = RFC(featuresCol='features', labelCol='rating', predictionCol='rating_pred', maxBins=1000, seed=42)

rfc_model = rfc.fit(df_train)

# gradient boosted tree classifier (training set)

from pyspark.ml.classification import GBTClassifier as GBTC

gbtc = GBTC(featuresCol='features', labelCol='rating', predictionCol='rating_pred', maxBins=1000, seed=42)

gbtc_model = gbtc.fit(df_train)


In [10]:
# model evaluation 

from pyspark.ml.evaluation import BinaryClassificationEvaluator as Evaluator

evaluator = Evaluator(labelCol='rating', rawPredictionCol='rating_pred')

# random forest classifier

rfc_train_pred = rfc_model.transform(df_train)
rfc_train_evaluation = evaluator.evaluate(rfc_train_pred)
print(f'Random Forest Classifier Accuracy (Train): {rfc_train_evaluation}')

rfc_test_pred = rfc_model.transform(df_test)
rfc_test_evaluation = evaluator.evaluate(rfc_test_pred)
print(f'Random Forest Classifier Accuracy (Test): {rfc_test_evaluation}\n')

# gradient boosted tree classifier 

gbtc_train_pred = gbtc_model.transform(df_train)
gbtc_train_evaluation = evaluator.evaluate(gbtc_train_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Train): {gbtc_train_evaluation}')

gbtc_test_pred = gbtc_model.transform(df_test)
gbtc_test_evaluation = evaluator.evaluate(gbtc_test_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Test): {gbtc_test_evaluation}\n')

Random Forest Classifier Accuracy (Train): 0.7922570130347584
Random Forest Classifier Accuracy (Test): 0.7926920781404597

Gradient Boosted Tree Classifier Accuracy (Train): 0.8335720671141592
Gradient Boosted Tree Classifier Accuracy (Test): 0.833173947483193

In [11]:
# confusion 'matrix' for  the random forest model (Train)
print('Confusion Matrix [Random Forest] (Train)')
rfc_train_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the random tree model (Train)
print('Confusion Matrix [Random Forest] (Test)')
rfc_test_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the gradient boosted tree model (Train)
print('Confusion Matrix [Gradient Boosted Tree] (Train)')
gbtc_train_pred.groupby('rating', 'rating_pred').count().show()

# confusion 'matrix' for  the gradient boosted tree model (Train)
print('Confusion Matrix [Gradient Boosted Tree] (Test)')
gbtc_test_pred.groupby('rating', 'rating_pred').count().show()

Confusion Matrix [Random Forest] (Train)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 93019|
 0| 0.0|655977|
 1| 1.0|275272|
 0| 1.0|127669|
+------+-----------+------+

Confusion Matrix [Random Forest] (Test)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 62057|
 0| 0.0|437105|
 1| 1.0|183821|
 0| 1.0| 84641|
+------+-----------+------+

Confusion Matrix [Gradient Boosted Tree] (Train)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 80250|
 0| 0.0|693560|
 1| 1.0|288041|
 0| 1.0| 90086|
+------+-----------+------+

Confusion Matrix [Gradient Boosted Tree] (Test)
+------+-----------+------+
rating|rating_pred| count|
+------+-----------+------+
 1| 0.0| 53723|
 0| 0.0|461663|
 1| 1.0|192155|
 0| 1.0| 60083|
+------+-----------+------+

In [12]:
rfc_pred = rfc_model.transform(df_ml)
# confusion 'matrix' for  the random forest model (Full)
print('\nConfusion Matrix [Random Forest]')
rfc_pred.groupby('rating', 'rating_pred').count().show()

# accuracy for entire dataset
rfc_pred =rfc_model.transform(df_ml)
rfc_evaluation = evaluator.evaluate(rfc_pred)
print(f'Random Forest Classifier Accuracy (Full): {rfc_evaluation}\n')


gbtc_pred = gbtc_model.transform(df_ml)
# confusion 'matrix' for  the gradient boosted tree model (Full)
print('\nConfusion Matrix [Gradient Boosted Tree]')
gbtc_pred.groupby('rating', 'rating_pred').count().show()

# accuracy for entire dataset
gbtc_pred = gbtc_model.transform(df_ml)
gbtc_evaluation = evaluator.evaluate(gbtc_pred)
print(f'Gradient Boosted Tree Classifier Accuracy (Full): {gbtc_evaluation}\n')


Confusion Matrix [Random Forest]
+------+-----------+-------+
rating|rating_pred| count|
+------+-----------+-------+
 1| 0.0| 155076|
 0| 0.0|1093082|
 1| 1.0| 459093|
 0| 1.0| 212310|
+------+-----------+-------+

Random Forest Classifier Accuracy (Full): 0.7924309612870397


Confusion Matrix [Gradient Boosted Tree]
+------+-----------+-------+
rating|rating_pred| count|
+------+-----------+-------+
 1| 0.0| 133973|
 0| 0.0|1155223|
 1| 1.0| 480196|
 0| 1.0| 150169|
+------+-----------+-------+

Gradient Boosted Tree Classifier Accuracy (Full): 0.8334127486181797

# Business Scenario

_In a situation, where a mobile advertisement company has this data as historical data. Each impression cost the advertisement company 1 cent, and each click costs the advertisement company $1. It has been given by merchants that for each impression the ROIfor the merchant is 10 cents and for each click, the ROI for the merchant is $10._

_The advertisement company has $1000 to run the advertisement campaigns in the next 7 days. 
Based on the above historical dataset, identify the {offerid, merchantid} combination (or combinations) that the advertisement agency should run in this campaign._

In [14]:
# starting dataframe for reference

df.show(10)

+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
 userid| offerid| category| merchant|rating|day|week|month|hour|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
b6208e4f5c0ea1a5d...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
b4dfe41dcb584aee3...|3ba463bbdb2e24eac...|100020213|10698b6475abd54c5...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
351163eb97a830c6c...|b75d6ee742849a622...|100020213|26467f203f42913b6...| 0| 4| 24| 6| 9|
d581f697c19367969...|be83df9772ec47fd2...| 125801|4740b6c83b6e12e42...| 0| 4| 24| 6| 9|
c1054eb52d2a47aec...|3735290a415dc236b...| 125801|8bf8f87492a799528...| 0| 4| 24| 6| 9|
8163ddf01acd57116...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
b4380e45e42d8a18d...|146d635197d4d9092...|100010713|66863da8db7e6c51b...| 0| 4| 24| 6| 9|
badefb50c4de15ff6...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...| 0| 4| 24| 6| 9|
+--------------------+--------------------+---------+--------------------+------+---+----+-----+----+
only showing top 10 rows

In [15]:
# Goal: We are trying to stay under advertising budget while attaining max ROI over a 7-day period
#
# Which merchant/offerid has best ROI for $1000 budget?


# calculate cost and roi

cost_vs_roi = spark.sql('''
select merchant, offerid, 
    concat(cast(week as string),'-',cast(day as string)) as day, 
    sum(rating) as clicks, count(offerid) as impressions,
    (count(offerid) * .01 + sum(rating)) as cost, 
    (count(offerid) + sum(rating) * 10) as roi
from impressions
group by offerid, merchant, week, day
order by day
''' )

cost_vs_roi.show(10)

cost_vs_roi.createOrReplaceTempView('cost_vs_roi')


+--------------------+--------------------+----+------+-----------+----+---+
 merchant| offerid| day|clicks|impressions|cost|roi|
+--------------------+--------------------+----+------+-----------+----+---+
05d1c767aa2e7558b...|b55c376d80dd40098...|24-1| 0| 1|0.01| 1|
00a3853a20be77f84...|e0f01ac6b7231f28d...|24-1| 0| 4|0.04| 4|
3acf8c2704776295a...|f112f25f7f7a1e9ba...|24-1| 1| 1|1.01| 11|
4caa35f6bf4466db6...|578fed60074b88bfc...|24-1| 1| 1|1.01| 11|
8bf8f87492a799528...|ca0af52dac608a62d...|24-1| 1| 1|1.01| 11|
a7b2f269064dbe77e...|8ecf5c62a03460635...|24-1| 0| 252|2.52|252|
ff1bd73f19ccc0984...|ab6b44b612ccabeba...|24-1| 1| 1|1.01| 11|
6a8e04db5c58a14bc...|04c575bbaf13259e7...|24-1| 1| 1|1.01| 11|
cde3ea6a59c6dfabc...|e7c6dac07c3625ebd...|24-1| 2| 2|2.02| 22|
a4df2039cb94bd95e...|70a869fd3bf61bb23...|24-1| 0| 1|0.01| 1|
+--------------------+--------------------+----+------+-----------+----+---+
only showing top 10 rows

In [16]:
# merchant/offerid combination

group_by_mod = spark.sql('''
select merchant, offerid, day, sum(cost) as total_cost, sum(roi) as total_roi
from cost_vs_roi
group by merchant, offerid, day
order by merchant, day, total_cost desc, total_roi desc
''')

group_by_mod.show(10)

group_by_mod.createOrReplaceTempView('group_by_mod')

+--------------------+--------------------+----+----------+---------+
 merchant| offerid| day|total_cost|total_roi|
+--------------------+--------------------+----+----------+---------+
002d1cfd4f3092ab0...|c9d54b65f6b1ba1e9...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|40984497df60e975e...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|b117430dd82c673a8...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|299a41bdda149a021...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|2001c998e0751a96b...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|cc75f91e3c3a91b16...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|9bbab1a79387abeb9...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|f311e8808fa42fcdb...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|4597ca4fccb51dafb...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|1d41feab66bb54ac0...|24-1| 1.01| 11|
+--------------------+--------------------+----+----------+---------+
only showing top 10 rows

In [17]:
# now we want to look at the cost/roi in a 7 day window

seven_days = spark.sql('''
select merchant, offerid, day,
    sum(total_cost) over (partition by merchant, offerid 
                    order by day 
                    rows between 3 preceding and 3 following) as 7day_cost,
    sum(total_roi) over (partition by merchant, offerid
                    order by day 
                    rows between 3 preceding and 3 following) as 7day_roi
from group_by_mod
order by merchant, day
''')

seven_days.show(10)

seven_days.createOrReplaceTempView('seven_days')

+--------------------+--------------------+----+---------+--------+
 merchant| offerid| day|7day_cost|7day_roi|
+--------------------+--------------------+----+---------+--------+
002d1cfd4f3092ab0...|e7126e0f6f4f71860...|24-1| 2.05| 25|
002d1cfd4f3092ab0...|e1f67c60bdb28c38c...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|f311e8808fa42fcdb...|24-1| 2.02| 22|
002d1cfd4f3092ab0...|eaece87327294557d...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|40984497df60e975e...|24-1| 1.01| 11|
002d1cfd4f3092ab0...|fc2c189f526eb11c4...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|b559d632c643c5f0d...|24-1| 3.03| 33|
002d1cfd4f3092ab0...|7a5442d67f6960ea3...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|0965bea9dbc78105d...|24-1| 0.01| 1|
002d1cfd4f3092ab0...|a2c9328a05a8d71e8...|24-1| 0.01| 1|
+--------------------+--------------------+----+---------+--------+
only showing top 10 rows

In [18]:
%sql

/* find the max roi while staying under budget */

select merchant, offerid, 7day_cost, 7day_roi
from seven_days
where 7day_cost < 1000
order by 7day_roi desc
limit 1

merchant,offerid,7day_cost,7day_roi
21a509189fb0875c3732590121ff3fc86da770b0628c1812e82f023ee2e0f683,19754ec121b3a99fff3967646942de67,789.58,78058
